<img src="logo.png">

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [ ]:
datos = datasets.load_boston()

In [ ]:
boston = pd.DataFrame(datos.data, columns=datos.feature_names)
boston["objetivo"] = datos.target

In [ ]:
boston.head()

En primer lugar vamos a evaluar los algoritmos que conocemos hasta ahora y compararlos con los distintos algoritmos de ensamblado.

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def rmse_cv(estimador, X, y):
    preds = estimador.predict(X)
    return rmse(y, preds)

In [ ]:
resultados = {}

In [ ]:
from sklearn.tree import DecisionTreeRegressor

estimador_arbol = DecisionTreeRegressor()
resultados["arbol"] = cross_val_score(estimador_arbol, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

from sklearn.linear_model import ElasticNet, Lasso, Ridge

estimador_elnet = ElasticNet()
resultados["elasticnet"] = cross_val_score(estimador_elnet, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

estimador_lasso = Lasso()
resultados["lasso"] = cross_val_score(estimador_lasso, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

estimador_ridge = Ridge()
resultados["ridge"] = cross_val_score(estimador_ridge, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados

### Bagging

Los algoritmos de Bagging (Bootstrap aggregating) funcionan entrenando varios estimadores base y cambiando los datos de entrenamiento para cada uno. En sklearn los algoritmos de ensamblado de modelos se encuentran en el submódulo `sklearn.ensemble`. En cuanto a Bagging, sklearn tiene una versión para problemas de regresión (`BaggingRegressor`) y otra para problemas de clasificación (`BaggingClassifier`).

In [ ]:
from sklearn.ensemble import BaggingRegressor, BaggingClassifier

In [ ]:
print(BaggingRegressor.__doc__)

`BaggingRegressor` utiliza árboles de decisión como estimador base por defecto, sin embargo podemos utilizar uno distinto mediante el parámetro `base_estimator`.

In [ ]:
estimador_bagging_10 = BaggingRegressor(n_estimators=10)
error_cv = cross_val_score(estimador_bagging_10, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados["bagging_arbol_10"] = error_cv
error_cv

Aumentar el número de estimadores base es una forma limitada pero sencilla de mejorar el funcionamiento del modelo

In [ ]:
estimador_bagging_100 = BaggingRegressor(n_estimators=100)
error_cv = cross_val_score(estimador_bagging_100, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados["bagging_arbol_100"] = error_cv
error_cv

In [ ]:
estimador_bagging_elnet = BaggingRegressor(n_estimators=100, base_estimator=ElasticNet())
error_cv = cross_val_score(estimador_bagging_elnet, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados["bagging_elnet"] = error_cv
error_cv

En su momento vimos que existe un tipo de arbol de decision completamente aleatorio (Extremely Randomized Trees) que deciden la particion en cada nodo al azar. Vemos que al agrupar muchos de estos estimadores que son débiles (aunque mejores que tirar una moneda al azar, ya que un árbol de decision aleatorio aún así aprende a separar los elementos), la varianza general se reduce ya que la que aporta un arbol se complementa con la del de al lado.

In [ ]:
from sklearn.tree import ExtraTreeRegressor

estimador_bagging_arbol_aleatorio = BaggingRegressor(n_estimators=100, base_estimator=ExtraTreeRegressor())
error_cv = cross_val_score(estimador_bagging_arbol_aleatorio, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados["bagging_extra_arbol"] = error_cv
error_cv

### Boosting

Los algoritmos de boosting intentan mejorar los estimadores base asignando pesos en funcion de su funcionamiento individual. El algoritmo clásico de boosting es `AdaBoost`, que se encuentra en sklearn como `AdaBoostRegressor` y `AdaBoostClassifier`.

In [ ]:
from sklearn.ensemble import AdaBoostRegressor,AdaBoostClassifier

In [ ]:
print(AdaBoostRegressor.__doc__)

In [ ]:
estimador_adaboost = AdaBoostRegressor(n_estimators=100)

error_cv = cross_val_score(estimador_adaboost, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados["adaboost_100"] = error_cv
error_cv

### Gradient Boosting (GBRT)

Otro algoritmo de Boosting es Gradient Boosting que a cada iteración usa el algoritmo de Descenso de Gradiente para cada iteración y así entrenar un estimador nuevo que minimiza la función de error (*loss function*) del modelo.

Scikit-learn implementa el algoritmo de (Gradient Boosted Regression Trees), que usa árboles de decisión como estimadores base, en [GradientBoostingRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) y [GradientBoostingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

Gradient Boosting puede usar cualquier funcion de error siempre que sea diferenciable.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier

In [ ]:
print(GradientBoostingRegressor.__doc__)

In [ ]:
estimador_gradientboost = GradientBoostingRegressor(n_estimators=100, loss='ls')

error_cv = cross_val_score(estimador_gradientboost, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados["gradientboost_100"] = error_cv
error_cv

Como cualquier estimador basado en árboles, `GradientBoostRegressor` nos permite ver la importancia de las variables en el modelo final.

In [ ]:
estimador_gradientboost.fit(boston[datos.feature_names], boston.objetivo)

importancia_variables = estimador_gradientboost.feature_importances_
importancia_variables = 100.0 * (importancia_variables / importancia_variables.max())
sorted_idx = np.argsort(importancia_variables)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.barh(pos, importancia_variables[sorted_idx], align='center')
plt.yticks(pos, datos.feature_names[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

### Bosques Aleatorios (Random Forests)

El algoritmo de Bosques Aleatorios funciona mediante la creación de árboles de decision entrenados en un subgrupo aleatorio de variables.

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [ ]:
print(RandomForestRegressor.__doc__)

La implementación de scikit-learn de RandomForest hace que cada árbol se entrene con base en un dataset del mismo tamaño que el original (con reemplazo si se usa la opción `bootstrap=True`).

En cuanto al criterio para evaluar la calidad de la separación de un node de cada árbol base, para la implementación de Regresion, `RandomForestRegressor` usa el error medio cuadrático `mse` por defecto.

In [ ]:
estimador_randomforest = RandomForestRegressor(n_estimators=100)

error_cv = cross_val_score(estimador_randomforest, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados["randomforest_100"] = error_cv
error_cv

In [ ]:
resultados

### XGBoost

XGBoost (eXtreme Gradient Boosting) es un algoritmo de boosting relativamente nuevo que tiene bastante acogida. Es una implementación de Gradient Boosted Trees pero enfocado a datasets grandes.

Al ser muy nuevo (el proyecto se creó en 2014 y el paper se publicó en 2016, [éste es el paper](https://arxiv.org/abs/1603.02754)) no está implementado en scikit-learn. Sin embargo existe en el paquete [xgboost](http://xgboost.readthedocs.io/en/latest/python/python_intro.html), que proporciona estimadores con base en dicho algoritmo que son compatibles con sklearn.

Podemos instalar `xgboost` de conda-forge

In [ ]:
#!pip install xgboost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
print(XGBRegressor.__doc__)

In [ ]:
estimador_xgboost = XGBRegressor(n_estimators=100)

error_cv = cross_val_score(estimador_xgboost, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados["xgboost_100"] = error_cv
error_cv

In [ ]:
from xgboost import plot_importance
estimador_xgboost.fit(boston[datos.feature_names], boston.objetivo)

Cuando tenemos un modelo entrenado, podemos ver la importancia de las variables en partir los árboles mediante el método `plot_importance`

In [ ]:
plot_importance(estimador_xgboost)

### Stacking

El algoritmo de stacking simplemente usa el output (generalmente en terminos de probabilidades para casos de clasificacion o de las predicciones en casos de regresión) de múltiples modelos como input para un nuevo *metamodelo*.

scikit learn no tiene un estimador de stacking por defecto, sin embargo, podemos usar el  estimador de stacking (`StackingRegressor`) de [mlxtend](https://rasbt.github.io/mlxtend/user_guide/regressor/StackingRegressor/), una librería que amplia las funcionalidades de `sklearn`

Podemos instalar mlxtend asi:

In [ ]:
from mlxtend.regressor import StackingRegressor

In [ ]:
print(StackingRegressor.__doc__)

Por ejemplo, podemos usar los estimadores ensamblados que hemos creado en este notebook para crear un nuevo estimador. Dicho estimador no tiene garantizado un funcionamiento mejor que el mejor de los estimadores que usa como input.

In [ ]:
estimador_stacking = StackingRegressor(
    regressors=[
        BaggingRegressor(n_estimators=100),
        AdaBoostRegressor(n_estimators=100),
        GradientBoostingRegressor(n_estimators=100),
        RandomForestRegressor(n_estimators=100)
    ], 
    meta_regressor=XGBRegressor(n_estimators=100))


error_cv = cross_val_score(estimador_stacking, X=boston[datos.feature_names], y=boston["objetivo"], 
                scoring=rmse_cv, cv=10).mean()

resultados["stacking"] = error_cv

error_cv

In [ ]:
resultados